In [27]:
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime
import json

In [4]:
consumer_key="TY6dPFQYUXduW9L3w5f36vd43"
consumer_secret="0nP8USJo2TgUsFSKc2eMvW1kAObgAKBADwhfKP5jMrAJB4ck98"
access_token="908811176-R9DIOWfODGjPWkxoUY1bvJ6733x3ArIN6xL0oJ6c"
access_token_secret="5lHiyp4XbDMaWnfNdDxBXRHJhVnbymjMMJLwHVH4Eudmq"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth, wait_on_rate_limit=True)

In [23]:
target = "ProFuturo_"

item = auth_api.get_user(target)

account_dict = {
                    "account_id" : item.id,
                    "account_username" : item.screen_name,
                    "account_full_name" : item.name,
                    "account_biography" : item.description,
                    "account_profile_picture_url" : item.profile_image_url_https,
                    "account_external_url" : item.url,
                    "account_n_posts" : item.statuses_count,
                    "account_followers" : item.followers_count,
                    "account_follows" : item.friends_count,
                    "account_is_private" : item.protected,
                    "account_is_verified" : item.verified
               }

if(item.location != None): 
    account_dict["location"] = item.location 
else:
    account_dict["location"] = ""



posts = []


for full_tweets in Cursor(auth_api.user_timeline,screen_name=target,timeout=999999).items(30):
    tweet_dict = {}
   
    tweet_dict["post_id"] = full_tweets.id
    tweet_dict["n_likes"] = full_tweets.favorite_count
    tweet_dict["n_retweets"] = full_tweets.retweet_count
    tweet_dict["caption"] = full_tweets.text
    tweet_dict["created_time"] = datetime.timestamp(full_tweets.created_at)
    
    replies=[]
    for tweet in Cursor(auth_api.search,q=f"to:{target}", result_type='recent', timeout=999999).items(100): 
        if hasattr(tweet, 'in_reply_to_status_id_str'): 
            if (tweet.in_reply_to_status_id_str==full_tweets.id_str): 
                status_to_dict = {
                                    "id_comment": tweet.id,
                                    "text": tweet.text,
                                    "owner": tweet.author.screen_name,
                                    "created_at":  datetime.timestamp(tweet.created_at)
                                 }
                author = auth_api.get_user(tweet.author.screen_name)
                
                if(item.location != None): 
                    status_to_dict["location"] = author.location 
                else:
                    status_to_dict["location"] = ""
                
                replies.append(status_to_dict)
    tweet_dict['comments'] = replies
    posts.append(tweet_dict)

account_dict["posts"] = posts

In [59]:
with open('corpus_twitter_posts.json', 'w', encoding='utf-8') as f:
    json.dump(account_dict, f, ensure_ascii=False, indent=4)